In [53]:
# In The Name of God - 14/12/00

# Dataset CelebA

In [54]:
#@title
# Imports
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import numpy as np
from collections import OrderedDict
import time

from torch.utils.data import random_split
from math import floor

import torchvision.utils as vutils
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

import itertools
import random

import shutil
from zipfile import ZipFile
import os

from IPython.core.debugger import set_trace

In [55]:
# Hyper parameters:
# change lastRunEpochNumber for every run based on saved files
isFirstRun = False
lastRunEpochNumber = 183
manual_seed = 20
image_size = 64
use_whole_dataset = True
usage_percent = 1.0
celeba_male_index = 20
celeba_young_index = 39
celeba_smiling_index = 31
celeba_straight_hair_index = 32
celeba_wavy_hair_index = 33
celeba_black_hair_index = 8
celeba_blonde_hair_index = 9
celeba_brown_hair_index = 11
celeba_gray_hair_index = 17
celeba_eyeglasses_index = 15
celeba_chubby_index = 13

celeba_mouth_open_index = 21
celeba_high_cheekbone_index = 19

using_index = celeba_smiling_index

learning_rate = 0.001 #0.2
batch_size = 1 #64

files_not_ready = True
dataset_folder_path = 'drive/MyDrive/ML/datasetShortcuts/CelebA'
#'C:/Workspace/Datasets/CelebA'
#'D:/on going/server/CelebA'
# dataset_folder_path = 'drive/MyDrive/ML/datasetShortcuts/CelebA'
data_dir = 'celeba'
save_dir = 'p2r-ae-9rev2-G-S'
saving_path = 'drive/MyDrive/jam4/' + save_dir + '/'
#'C:/Users/jamshidi/saveMLs/' + save_dir + '/'
#'C:/Users/Mohammad/savedML/' + save_dir + '/'
# saving_path = 'drive/MyDrive/ML/saves/' + save_dir + '/'

# creating modified dataset:
m_version = '-nog-awgn-35'
on_drive_modified_dataset_saving_path = 'drive/MyDrive/ML/modifiedDatasets/' + save_dir + m_version + '/'
#'C:/Users/jamshidi/modifiedDatasets/' + save_dir + m_version + '/'
#'C:/Users/Mohammad/modifiedDatasets/' + save_dir + m_version + '/'
on_fly_modified_dataset_saving_path = 'modifiedDatasets/' + save_dir + m_version + '/'


# saving_path = 'drive/MyDrive/ML/modifiedDatasets/' + save_dir + m_version + '/'
use_g = False
g_eff_val = -3000
miu = 0
coef_for_var = 35
load_m_dataset = True
create_zip_from_m_dataset = True
delete_m_dataset_folder = False
zip_and_copy_to_drive = True

suffling_main_train_data = False
suffling_modified_train_data = False

# use_early_stop = False
# use_rate_schedule = False
# use_dis_acc_stopper = True

# Other params
# Number of workers for dataloader
workers = 2
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1
# Size of feature maps in encoder
nef = 64
# Size of feature maps in decoder
ndf = 64
# Number of channels in the training images. For color images this is 3
nc = 3
# Size of z latent vector (i.e. size of generator input)
nz = 100
# Number of training epochs
num_epochs = lastRunEpochNumber #100

In [56]:
# Check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [57]:
# Mount google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:

# download dataset and unzip
# data_dir variable: data_dir = 'img_align_celeba'
dataset_zip_path = dataset_folder_path + '/Img/img_align_celeba.zip'
list_eval_partition_path = dataset_folder_path + '/Eval/list_eval_partition.txt'
identity_celeba_path = dataset_folder_path + '/Anno/identity_CelebA.txt'
list_attr_celeba_path = dataset_folder_path + '/Anno/list_attr_celeba.txt'
list_bbox_celeba_path = dataset_folder_path + '/Anno/list_bbox_celeba.txt'
list_landmarks_align_celeba_path = dataset_folder_path + '/Anno/list_landmarks_align_celeba.txt'

if files_not_ready:
    try:
      os.mkdir(data_dir)
      print("data folder created successfully")
    except OSError as e:
      print("Error: %s" % (e.strerror))

    shutil.copyfile(dataset_zip_path, data_dir + r'/img_align_celeba.zip')
    shutil.copyfile(list_eval_partition_path, data_dir + r'/list_eval_partition.txt')
    shutil.copyfile(identity_celeba_path, data_dir + r'/identity_CelebA.txt')
    shutil.copyfile(list_attr_celeba_path, data_dir + r'/list_attr_celeba.txt')
    shutil.copyfile(list_bbox_celeba_path, data_dir + r'/list_bbox_celeba.txt')
    shutil.copyfile(list_landmarks_align_celeba_path, data_dir + r'/list_landmarks_align_celeba.txt')

    try:
        shutil.rmtree(data_dir + r'/img_align_celeba')
        print("old unzipped directory removed successfully")
    except OSError as e:
        print("Error: %s" % (e.strerror))

    archive = data_dir + r'/img_align_celeba.zip'
    with ZipFile(archive, 'r') as zip:
       zip.extractall(data_dir)
try:
    os.mkdir(saving_path)
    print("saving_path directory created successfully")
except OSError as e:
    print("Error: %s" % (e.strerror))

Error: File exists
old unzipped directory removed successfully
Error: File exists


In [59]:
#@title
# Class - Learning Rate Scheduler
class LRScheduler():
    """
    Learning rate scheduler. If the validation loss does not decrease for the
    given number of `patience` epochs, then the learning rate will decrease by
    by given `factor`.
    """
    def __init__(
        self, optimizer, patience=5, min_lr=1e-6, factor=0.5
    ):
        """
        new_lr = old_lr * factor
        :param optimizer: the optimizer we are using
        :param patience: how many epochs to wait before updating the lr
        :param min_lr: least lr value to reduce to while updating
        :param factor: factor by which the lr should be updated
        """
        self.optimizer = optimizer
        self.patience = patience
        self.min_lr = min_lr
        self.factor = factor
        self.lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer,
                mode='min',
                patience=self.patience,
                factor=self.factor,
                min_lr=self.min_lr,
                verbose=True
            )
    def __call__(self, val_loss):
        self.lr_scheduler.step(val_loss)

In [60]:
#@title
# Class - Early Stopping
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=10, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
    def __call__(self, score):
        if self.best_score == None:
            self.best_score = score
        elif self.best_score - score > self.min_delta:
            self.best_score = score
            self.counter = 0
        elif self.best_score - score < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [61]:
#@title
# Class - Discriminator accuracy stopper
class DisAccStopping():
    """
    Discriminator accuracy stopper to stop the training when the discriminator accuracy closes to 50%
    """
    def __init__(self, patience=5, min_delta=2):
        """
        :param patience: how many epochs to wait before stopping when accuracy is
               remain near 50%
        :param min_delta: minimum difference between accuracy and 50%
               to be considered as closeness
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        #self.best_score = None
        self.dis_acc_stop = False
    def __call__(self, score):
        if abs(score - 50) >= self.min_delta:
            self.counter = 0
        elif abs(score - 50) < self.min_delta:
            self.counter += 1
            print(f"INFO: Discriminator accuracy stopper counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Discriminator accuracy stopping')
                self.dis_acc_stop = True

In [62]:
#@title
# Class - oneFeatureSampler

from typing import Iterator, Iterable, Optional, Sequence, List, TypeVar, Generic, Sized, Union

class OneFeatureSampler(torch.utils.data.Sampler[int]):
    def __init__(self, mask) -> None:
        self.mask = mask
        self.indices = torch.nonzero(mask)
    def __iter__(self) -> Iterator[int]:
        for i in range(len(self.indices)):
            yield self.indices[i]
    def __len__(self) -> int:
        return len(self.indices)

In [63]:

# Define transforms
train_transforms = transforms.Compose([#transforms.RandomRotation(30),
                                       #transforms.RandomResizedCrop(224),
                                       #transforms.RandomHorizontalFlip(),
                                       transforms.Resize(image_size),
                                       transforms.CenterCrop(image_size),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(image_size),
                                      transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [64]:
#@title
# Function - use some percent of data
def shorten_dataset(dataset, usage_percent=1.0):
  len_used = floor(len(dataset)*usage_percent)
  len_not_used = len(dataset) - len_used
  used_dataset, not_used_dataset = random_split(dataset, [len_used, len_not_used], generator=torch.Generator().manual_seed(manual_seed))
  return used_dataset

In [65]:
#@title
# Function - Split Data to Public & Private
def split_to_public_private(dataset, usage_percent=1.0, public_percent=0.25):
  len_used = floor(len(dataset)*usage_percent)
  len_not_used = len(dataset) - len_used
  used_dataset, not_used_dataset = random_split(dataset, [len_used, len_not_used], generator=torch.Generator().manual_seed(manual_seed))
  len_public = floor(len(used_dataset)*public_percent)
  len_private = len(used_dataset) - len_public
  return random_split(used_dataset, [len_public, len_private], generator=torch.Generator().manual_seed(manual_seed))


In [66]:
#@title
# Function - Specialized DataLoader
def init_specialized_dataLoader(dataset, label, label_val, batch_size):
  if type(dataset) == torch.utils.data.dataset.Subset:
    mask = [1 if dataset.dataset.attr[:,label][i] == label_val else 0 for i in dataset.indices]
  else:
    mask = [1 if dataset.attr[:,label][i] == label_val else 0 for i in range(len(dataset))]
  mask = torch.tensor(mask)
  sampler = OneFeatureSampler(mask)
  return torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler = sampler, shuffle=False, num_workers=workers, drop_last=True)

In [67]:

# Load Datas
train_set = datasets.CelebA(root='', download=False, split='train', target_type=["attr", "identity"], transform=train_transforms)
test_set = datasets.CelebA(root='', download=False, split='test', target_type=["attr", "identity"], transform=test_transforms)
valid_set = datasets.CelebA(root='', download=False, split='valid', target_type=["attr", "identity"], transform=test_transforms)

# shorten Dataset
if not use_whole_dataset:
  train_set = shorten_dataset(train_set, usage_percent)
  test_set = shorten_dataset(test_set, usage_percent)
  valid_set = shorten_dataset(valid_set, usage_percent)

# DataLoader
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=suffling_main_train_data, num_workers=workers)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=workers)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, num_workers=workers)


In [68]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [69]:
# # Plot some images with corresponding labels
# first_batch = next(iter(train_loader))
# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.title("Training Images")
# plt.imshow(np.transpose(vutils.make_grid(first_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
# # if use_whole_dataset:
# #     print((first_batch[1][0])[:,0,39])
# # else:
# print((first_batch[1][0])[:,39])

In [70]:
#@title
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [71]:
# Encoder Model
class Encoder(nn.Module):
    def __init__(self, ngpu):
        super(Encoder, self).__init__()
        self.ngpu = ngpu

        # input is nc x 64 x 64
        self.conv1 = nn.Conv2d(nc, nef, 4, 2, 1, bias=False)
        self.actv1 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 32 x 32
        self.conv2 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor2 = nn.BatchNorm2d(nef)
        self.actv2 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 16 x 16
        self.conv3 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor3 = nn.BatchNorm2d(nef)
        self.actv3 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 8 x 8
        self.conv4 = nn.Conv2d(nef, nef * 2, 4, 2, 1, bias=False)
        self.bnor4 = nn.BatchNorm2d(nef * 2)
        self.actv4 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef*2) x 4 x 4
        # shaping would be here: nef*2 x 4 x 4 -> 2048
        # state size. 2048
        self.fllc5 = nn.Linear(nef*2*4*4, nef*1*4*4)
        self.actv5 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 1024

        # split features: 1024 -> 1022 + 2 (male/female)
        # first classifier: (gender)
        self.fllc_male_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_male_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_male_features1 = nn.Dropout(p=0.5)
        self.fllc_male_features2 = nn.Linear(nef*1*4*4, nef*4)
        self.actv_male_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_male_features2 = nn.Dropout(p=0.5)
        self.fllc_male_features3 = nn.Linear(nef*4, nef)
        self.actv_male_features3 = nn.LeakyReLU(0.2, inplace=True)
        self.fllc_male_features4 = nn.Linear(nef, 2)
        self.actv_male_features4 = nn.LogSoftmax(dim=1)
        # second classifier: (hair color)
        # self.fllc_hair_color_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        # self.actv_hair_color_features1 = nn.LeakyReLU(0.2, inplace=True)
        # self.dropout_hair_color_features1 = nn.Dropout(p=0.5)
        # self.fllc_hair_color_features2 = nn.Linear(nef*1*4*4, nef*4)
        # self.actv_hair_color_features2 = nn.LeakyReLU(0.2, inplace=True)
        # self.dropout_hair_color_features2 = nn.Dropout(p=0.5)
        # self.fllc_hair_color_features3 = nn.Linear(nef*4, nef)
        # self.actv_hair_color_features3 = nn.LeakyReLU(0.2, inplace=True)
        # self.fllc_hair_color_features4 = nn.Linear(nef, 6)
        # self.actv_hair_color_features4 = nn.LogSoftmax(dim=1)
        # other features
        self.fllc_other_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_other_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_other_features1 = nn.Dropout(p=0.5)
        self.fllc_other_features2 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_other_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_other_features2 = nn.Dropout(p=0.5)
        self.fllc_other_features3 = nn.Linear(nef*1*4*4, nef*1*4*4 - 2)
        self.actv_other_features3 = nn.LeakyReLU(0.2, inplace=True)
        # aggregate features for output

    def forward(self, x):
        # Part 1:
        x = self.conv1(x)
        x = self.actv1(x)
        x = self.conv2(x)
        x = self.bnor2(x)
        x = self.actv2(x)
        x = self.conv3(x)
        x = self.bnor3(x)
        x = self.actv3(x)
        x = self.conv4(x)
        x = self.bnor4(x)
        x = self.actv4(x)
        # flatten
        x = torch.flatten(x, start_dim = 1)
        # Part 2:
        x = self.fllc5(x)
        x = self.actv5(x)
        # first classifier: (gender)
        y1 = self.fllc_male_features1(x)
        y1 = self.actv_male_features1(y1)
        y1 = self.dropout_male_features1(y1)
        y1 = self.fllc_male_features2(y1)
        y1 = self.actv_male_features2(y1)
        y1 = self.dropout_male_features2(y1)
        y1 = self.fllc_male_features3(y1)
        y1 = self.actv_male_features3(y1)
        y1 = self.fllc_male_features4(y1)
        y1 = self.actv_male_features4(y1)
        # second classifier: (hair color)
        # y2 = self.fllc_hair_color_features1(x)
        # y2 = self.actv_hair_color_features1(y2)
        # y2 = self.dropout_hair_color_features1(y2)
        # y2 = self.fllc_hair_color_features2(y2)
        # y2 = self.actv_hair_color_features2(y2)
        # y2 = self.dropout_hair_color_features2(y2)
        # y2 = self.fllc_hair_color_features3(y2)
        # y2 = self.actv_hair_color_features3(y2)
        # y2 = self.fllc_hair_color_features4(y2)
        # y2 = self.actv_hair_color_features4(y2)
        # other features
        y3 = self.fllc_other_features1(x)
        y3 = self.actv_other_features1(y3)
        y3 = self.dropout_other_features1(y3)
        y3 = self.fllc_other_features2(y3)
        y3 = self.actv_other_features2(y3)
        y3 = self.dropout_other_features2(y3)
        y3 = self.fllc_other_features3(y3)
        y3 = self.actv_other_features3(y3)
        return y1, y3

In [72]:
# Decoder Model
class Decoder(nn.Module):
    def __init__(self, ngpu):
        super(Decoder, self).__init__()
        self.ngpu = ngpu

        # input size is 1024
        self.fllc6 = nn.Linear(nef*1*4*4, ndf*2*4*4)
        self.actv6 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 2048
        # shaping would be here: 2048 -> ndf*2 x 4 x 4
        # state size. (ndf*2) x 4 x 4
        self.cnvt7 = nn.ConvTranspose2d( ndf*2, ndf, 4, 2, 1, bias=False)
        self.bnor7 = nn.BatchNorm2d(ndf)
        self.actv7 = nn.ReLU(True)
        # state size. (ndf) x 8 x 8
        self.cnvt8 = nn.ConvTranspose2d(ndf, ndf, 4, 2, 1, bias=False)
        self.bnor8 = nn.BatchNorm2d(ndf)
        self.actv8 = nn.ReLU(True)
        # state size. (ndf) x 16 x 16
        self.cnvt9 = nn.ConvTranspose2d( ndf, ndf, 4, 2, 1, bias=False)
        self.bnor9 = nn.BatchNorm2d(ndf)
        self.actv9 = nn.ReLU(True)
        # state size. (ndf) x 32 x 32
        self.cnvt10 = nn.ConvTranspose2d( ndf, nc, 4, 2, 1, bias=False)
        self.actv10 = nn.Sigmoid() # nn.Tanh()
        # state size. (nc) x 64 x 64

    def forward(self, x):
        x = self.fllc6(x)
        x = self.actv6(x)
        x = x.view(batch_size, ndf*2, 4, 4) # 64 is batch_size
        x = self.cnvt7(x)
        x = self.bnor7(x)
        x = self.actv7(x)
        x = self.cnvt8(x)
        x = self.bnor8(x)
        x = self.actv8(x)
        x = self.cnvt9(x)
        x = self.bnor9(x)
        x = self.actv9(x)
        x = self.cnvt10(x)
        x = self.actv10(x)
        return x

In [73]:
# # AE Model
# class AEModel(nn.Module):
#     def __init__(self, ngpu):
#         super(AEModel, self).__init__()
#         self.ngpu = ngpu

#         self.encoder = Encoder(ngpu).to(device)
#         self.decoder = Decoder(ngpu).to(device)

#     def forward(self, x):
#         y1, y2, y3 = self.encoder(x)
#         y = torch.cat((y1, y2, y3), 1)
#         x = self.decoder(y)
#         return x, y1, y2

In [74]:
# AE Model + Noise
class AEModel(nn.Module):
    def __init__(self, ngpu, mode='train', miu=0, coef_for_var=0, g_eff_val=-10000):
        super(AEModel, self).__init__()
        self.ngpu = ngpu
        # self.change_mode(mode)
        # self.tune_noise(miu, coef_for_var)
        self.g_eff_val = g_eff_val
        self.miu = miu
        self.coef_for_var = coef_for_var
        self.mode = mode
        self.encoder = Encoder(ngpu).to(device)
        self.decoder = Decoder(ngpu).to(device)

    def tune_noise(self, miu=0, coef_for_var=0, g_eff_val=-10000):
        self.miu = miu
        self.coef_for_var = coef_for_var
        self.g_eff_val = g_eff_val

    def change_mode(self, mode='train'):
        self.mode = mode

    def add_noise(self, nodes):
      with torch.no_grad():
        var = (self.coef_for_var) * (torch.mean(nodes).item())
        noise = self.miu + (var) * torch.randn(nodes.size())
        noise = noise.to(device)
        nodes.add_(noise)
        return nodes

    def change_lbl(self, nodes, lbls):
      with torch.no_grad():
        lbls[lbls == 0] = self.g_eff_val
        lbls[lbls == 1] = 0
        nodes = lbls
        return nodes

    def forward(self, x, y1_real_lbl=[]):
        y1, y3 = self.encoder(x)
        if self.mode=='use':
            if use_g:
              # y1 = y1_real_lbl
              y1 = self.change_lbl(y1, y1_real_lbl)
            y3 = self.add_noise(y3)
        y = torch.cat((y1, y3), 1)
        x = self.decoder(y)
        return x, y1

In [75]:
# Create the AE
netAE = AEModel(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netAE = nn.DataParallel(netAE, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netAE.apply(weights_init)

# Print the model
print(netAE)

AEModel(
  (encoder): Encoder(
    (conv1): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (actv1): LeakyReLU(negative_slope=0.2, inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (bnor2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (actv2): LeakyReLU(negative_slope=0.2, inplace=True)
    (conv3): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (bnor3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (actv3): LeakyReLU(negative_slope=0.2, inplace=True)
    (conv4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (bnor4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (actv4): LeakyReLU(negative_slope=0.2, inplace=True)
    (fllc5): Linear(in_features=2048, out_features=1024, bias=True)
    (actv5): LeakyReLU(negati

In [76]:
# total parameters and trainable parameters
total_params = sum(p.numel() for p in netAE.parameters())
print(f"{total_params:,} training parameters.")

9,204,032 training parameters.


In [77]:
#@title
# Function - Save:
def save_model(name, number, model, res):
  checkpoint = {'res': res,
                'state_dict': model.state_dict()}
  torch.save(checkpoint, saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth')
  return True

In [78]:
#@title
# Function - Load:
def load_model(name, number, model, device):

  checkpoint = torch.load(saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth', map_location=device)
  res = checkpoint['res']
  model.load_state_dict(checkpoint['state_dict'])
  return {'model':model,
          'res':res}

In [79]:

# Function - Delete file:
def delete_file(name, number):
  address = saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth'
  try:
    os.remove(address)
    print("old file removed successfully")
  except OSError as e:
    print("Error: %s" % (e.strerror))

In [80]:
# Save Start Checkpoint
if(isFirstRun):
  res = {'train_losses': [],
         'valid_losses': [],
         'y1_train_losses': [],
         'y1_valid_losses': [],
        #  'y2_train_losses': [],
        #  'y2_valid_losses': [],
         'test_mse': [],
         'test_psnr': [],
         'test_y1_acc': [],
        #  'test_y2_acc': [],
         'epoch_number': 0
        };
  save_model('ae', 0, netAE, res)

In [81]:
# Load Last Checkpoint:
ae_load = load_model('ae', lastRunEpochNumber, netAE, device)

train_losses = ae_load['res']['train_losses']
valid_losses = ae_load['res']['valid_losses']
y1_train_losses = ae_load['res']['y1_train_losses']
y1_valid_losses = ae_load['res']['y1_valid_losses']
# y2_train_losses = ae_load['res']['y2_train_losses']
# y2_valid_losses = ae_load['res']['y2_valid_losses']
test_mse = ae_load['res']['test_mse']
test_psnr = ae_load['res']['test_psnr']
test_y1_acc = ae_load['res']['test_y1_acc']
# test_y2_acc = ae_load['res']['test_y2_acc']
last_epoch = ae_load['res']['epoch_number']

In [82]:
# test code:
# first_batch = next(iter(test_loader))
# inputs, labels = first_batch[0], first_batch[1]
# inputs = inputs.to(device)
# gender_target = extract_targets(labels)
# netAE.change_mode('use')
# do_reverse = True
# netAE.tune_noise(miu=0, coef_for_var=0, g_eff_val=-100)
# if do_reverse:
#   gender_target[gender_target == 0] = 1
#   gender_target[gender_target == 1] = 0
# output, y1 = netAE.forward(inputs, gender_target)

# plt.figure(figsize=(2,2))
# plt.axis("off")
# plt.title("Images")
# plt.imshow(np.transpose(vutils.make_grid(output.to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [83]:
#@title
# either early stopping or learning rate scheduler or discriminator accuracy stopper
# if use_rate_schedule:
#     print('INFO: Learning rate scheduler is on')
#     lr_scheduler = LRScheduler(optimizer)

# if use_early_stop:
#     print('INFO: Early stopping is on')
#     early_stopping = EarlyStopping()

# if use_dis_acc_stopper:
#     print('INFO: Discriminator Accuracy stopping is on')
#     dis_acc_stopper = DisAccStopping()


In [84]:
#@title
# Function - Random index for batch from loader
def rand_batch(before_loop, ref_iter):
  if before_loop:
    rand = random.randint(0, 40)
  else:
    rand = random.randint(1, 2)
  for i in range(rand):
    batch = next(ref_iter)
  return batch

In [85]:
def fix_getting_data_problem(labels, decision_label):
    if use_whole_dataset:
        # return labels[0][:, 0, decision_label]
        return labels[0][:, decision_label]
    else:
        return labels[0][:, decision_label]

In [86]:
def extract_targets(labels):
    male_labels = fix_getting_data_problem(labels, using_index)
    female_labels = torch.add(1, -male_labels)
    # HERE! there were a bug. we should concat(female,male) not concat(male,female)
    # gender_target = torch.cat((male_labels.view(batch_size,1), female_labels.view(batch_size,1)), 1).float() # 64 is batch_size
    gender_target = torch.cat((female_labels.view(batch_size,1), male_labels.view(batch_size,1)), 1).float() # 64 is batch_size
    gender_target = gender_target.to(device)
    return gender_target

In [87]:
# Utility for test & ...
# 1. breakpoint
# from IPython.core.debugger import Pdb; Pdb().set_trace()

# 2. attributes & pics:
## first index:
# pic_index = 0
## second index:
# pic = 0
# labels = 1
## third index:
# attributes = 0
# identity = 1
## usage:
# test_loader.dataset[pic_index][labels][attributes]

# 3. check result of decision on modified dataset (utility) by obfuscator model
# torch.set_printoptions(precision=1, sci_mode=False)

# a,at=torch.exp(y1).topk(1, dim=1)
# equals = at == gender_label_top_class
# print(equals.sum().item())

# ps_y2 = torch.exp(y2)
# top_p_y2_1, top_class_y2_1 = ps_y2[:,0:2].topk(1, dim=1)
# top_p_y2_2, top_class_y2_2 = ps_y2[:,2:4].topk(1, dim=1)
# top_p_y2_3, top_class_y2_3 = ps_y2[:,4:6].topk(1, dim=1)

# c1, ct1 = predefined_lbl_y2[:,0:2].topk(1, dim=1)
# c2, ct2 = predefined_lbl_y2[:,2:4].topk(1, dim=1)
# c3, ct3 = predefined_lbl_y2[:,4:6].topk(1, dim=1)

# equals_y2_1 = top_class_y2_1 == ct1
# equals_y2_2 = top_class_y2_2 == ct2
# equals_y2_3 = top_class_y2_3 == ct3

# print(equals_y2_1.sum().item())
# print(equals_y2_2.sum().item())
# print(equals_y2_3.sum().item())

In [88]:
# Create new dataset folder:

m_celeba_dir = on_fly_modified_dataset_saving_path + 'celeba/'
img_celeba_dir = on_fly_modified_dataset_saving_path + 'celeba/img_align_celeba/'
try:
    os.mkdir('modifiedDatasets')
except OSError as e:
    print("Error: %s" % (e.strerror))
try:
    os.mkdir(on_fly_modified_dataset_saving_path)
except OSError as e:
    print("Error: %s" % (e.strerror))
try:
    os.mkdir(m_celeba_dir)
except OSError as e:
    print("Error: %s" % (e.strerror))
try:
    os.mkdir(img_celeba_dir)
except OSError as e:
    print("Error: %s" % (e.strerror))

try:
    shutil.copyfile(list_eval_partition_path, m_celeba_dir + r'list_eval_partition.txt')
    shutil.copyfile(identity_celeba_path, m_celeba_dir + r'identity_CelebA.txt')
    shutil.copyfile(list_attr_celeba_path, m_celeba_dir + r'list_attr_celeba.txt')
    shutil.copyfile(list_bbox_celeba_path, m_celeba_dir + r'list_bbox_celeba.txt')
    shutil.copyfile(list_landmarks_align_celeba_path, m_celeba_dir + r'list_landmarks_align_celeba.txt')
    print("label files copied successfully")
except OSError as e:
    print("Error: %s" % (e.strerror))

Error: File exists
label files copied successfully


In [89]:
# Function save image:
from torchvision.utils import save_image

def store_single_disk(image, file_name):
    # save as png:
    # file_name = file_name.replace('.jpg', '.png')
    save_image(image, img_celeba_dir + file_name)


In [90]:
def convert_dataset(my_loader):
    # counter = 0
    prog_bar = tqdm(enumerate(my_loader), total=len(my_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            gender_target = extract_targets(labels)
            output, y1 = netAE.forward(inputs, gender_target)
            for j in range(batch_size):
                store_single_disk(output[j,:,:,:], my_loader.dataset.filename[i*batch_size + j])
            # if counter >= 10:
            #     break
            # counter += 1

In [91]:
# using model to convert public to reference dataset (using test dataloader)
# using: y1 (gender) - replace by real label
#        y2 (hair color) - changed to fix value (0,0,1,0) - brown
#        y3 (others) - adding noise - N(0,1)
netAE.change_mode('use')
netAE.eval()
netAE.tune_noise(miu, coef_for_var, g_eff_val)
# predefined_lbl_y2 = torch.tensor([1, 0, 0, 0, 0, 0]).repeat(batch_size, 1).to(device)
print("Converting train images from public to references...\n")
convert_dataset(train_loader)
print("\nConverting valid images from public to references...")
convert_dataset(valid_loader)
print("\nConverting test images from public to references...")
convert_dataset(test_loader)

Converting train images from public to references...



100%|██████████| 162770/162770 [17:10<00:00, 158.02it/s]


Converting valid images from public to references...



100%|██████████| 19867/19867 [02:05<00:00, 158.29it/s]


Converting test images from public to references...



100%|██████████| 19962/19962 [02:06<00:00, 157.72it/s]


In [92]:
# load modified dataset:
if load_m_dataset:
    m_transforms = transforms.Compose([
                                      #transforms.Resize(image_size),
                                      #transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      #transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                      ])
    m_train_set = datasets.CelebA(root=on_fly_modified_dataset_saving_path, download=False, split='train', target_type=["attr", "identity"], transform=m_transforms)
    m_valid_set = datasets.CelebA(root=on_fly_modified_dataset_saving_path, download=False, split='valid', target_type=["attr", "identity"], transform=m_transforms)
    m_test_set = datasets.CelebA(root=on_fly_modified_dataset_saving_path, download=False, split='test', target_type=["attr", "identity"], transform=m_transforms)
    m_train_loader = torch.utils.data.DataLoader(m_train_set, batch_size=batch_size, shuffle=suffling_modified_train_data, num_workers=workers, drop_last=True)
    m_valid_loader = torch.utils.data.DataLoader(m_valid_set, batch_size=batch_size, num_workers=workers, drop_last=True)
    m_test_loader = torch.utils.data.DataLoader(m_test_set, batch_size=batch_size, num_workers=workers, drop_last=True)


In [93]:
# zip new pics folder & delete other files
if create_zip_from_m_dataset:
    shutil.make_archive(on_fly_modified_dataset_saving_path + 'img_align_celeba', 'zip', img_celeba_dir)

if delete_m_dataset_folder:
    try:
        shutil.rmtree(m_celeba_dir)
        print("modified dataset directory removed successfully")
    except OSError as e:
        print("Error: %s" % (e.strerror))

In [94]:
# zip celeba folder and copy to drive
if zip_and_copy_to_drive:
  try:
    os.mkdir(on_drive_modified_dataset_saving_path)
  except OSError as e:
    print("Error: %s" % (e.strerror))
  zipped_address = shutil.make_archive(on_drive_modified_dataset_saving_path + 'celeba', 'zip', m_celeba_dir)

In [95]:
# first_batch = next(iter(train_loader))
# plt.figure(figsize=(2,2))
# plt.axis("off")
# plt.title("Images")
# plt.imshow(np.transpose(vutils.make_grid(first_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [96]:
# m_first_batch = next(iter(m_train_loader))
# plt.figure(figsize=(2,2))
# plt.axis("off")
# plt.title("Images")
# plt.imshow(np.transpose(vutils.make_grid(m_first_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [97]:
# # Training Loop
# netAE.to(device)
# calc_every_epoch = 1
# save_every_epoch = 1

# start = time.time()
# print("Starting Training Loop...")

# for epoch in range(last_epoch+1, num_epochs+1):
#     print(f"Epoch {epoch}/{num_epochs}: ")
#     train_loss, y1_train_loss, y2_train_loss = fit(netAE, train_loader, optimizer, criterion)
#     valid_loss, y1_valid_loss, y2_valid_loss = validate(netAE, valid_loader, optimizer, criterion)
#     mse_loss, psnr_loss, y1_accuracy, y2_accuracy = calcAccuracyTest(netAE, test_loader)

#     train_losses.append(train_loss)
#     valid_losses.append(valid_loss)
#     y1_train_losses.append(y1_train_loss)
#     y1_valid_losses.append(y1_valid_loss)
#     y2_train_losses.append(y2_train_loss)
#     y2_valid_losses.append(y2_valid_loss)
#     test_mse.append(mse_loss)
#     test_psnr.append(psnr_loss)
#     test_y1_acc.append(y1_accuracy)
#     test_y2_acc.append(y2_accuracy)

#     res = {'train_losses': train_losses,
#            'valid_losses': valid_losses,
#            'y1_train_losses': y1_train_losses,
#            'y1_valid_losses': y1_valid_losses,
#            'y2_train_losses': y2_train_losses,
#            'y2_valid_losses': y2_valid_losses,
#            'test_mse': test_mse,
#            'test_psnr': test_psnr,
#            'test_y1_acc': test_y1_acc,
#            'test_y2_acc': test_y2_acc,
#            'epoch_number': epoch
#           };
#     if epoch % save_every_epoch == 0:
# #       if epoch-save_every_epoch>=0:
# #         delete_file('obf', epoch-save_every_epoch)
# #         delete_file('dis', epoch-save_every_epoch)
#       save_model('ae', epoch, netAE, res)

#     # if use_dis_acc_stopper:
#     #   dis_acc_stopper(dis_valid_epoch_accuracy)
#     #   if dis_acc_stopper.dis_acc_stop:
#     #     break

#     print(f"Train Loss: {train_loss:.2f}")
#     print(f"Valid Loss: {valid_loss:.2f}")
#     print(f"gender Train Loss: {y1_train_loss:.2f}")
#     print(f"gender Valid Loss: {y1_valid_loss:.2f}")
#     print(f"hair color Train Loss: {y2_train_loss:.2f}")
#     print(f"hair color Valid Loss: {y2_valid_loss:.2f}")

# end = time.time()
# print(f"Training time: {(end-start)/60:.3f} minutes")

# print('TRAINING COMPLETE')

In [98]:
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'

# print('Loss plot...')

# # loss plots
# plt.figure(figsize=(10,7))
# plt.title("Train-Valid Loss Trend")
# plt.plot(train_losses, color='green', label='Training Loss')
# plt.plot(valid_losses, color='blue', label='Validation Loss')
# plt.plot(test_mse, color='red', label='MSE Test Set')
# plt.legend(frameon=False)
# plt.xlabel("epochs")
# plt.ylabel("Loss")
# plt.savefig(saving_path + "loss_plot.png")
# plt.show()

In [99]:
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'

# print('psnr plot...')

# # loss plots
# plt.figure(figsize=(10,7))
# plt.title("PSNR Test Set Trend")
# plt.plot(test_psnr, color='red', label='PSNR Test Set')
# plt.legend(frameon=False)
# plt.xlabel("epochs")
# plt.ylabel("Loss")
# plt.savefig(saving_path + "psnr_test_plot.png")
# plt.show()

In [100]:
# plt.figure(figsize=(10,7))
# plt.title("Gender Train-Valid Loss Trend")
# plt.plot(y1_train_losses, color='green', label='Gender Training Loss')
# plt.plot(y1_valid_losses, color='blue', label='Gender Validation Loss')
# plt.legend(frameon=False)
# plt.xlabel("epochs")
# plt.ylabel("Gender Loss")
# plt.savefig(saving_path + "gender_loss_plot.png")
# plt.show()

In [101]:
# plt.figure(figsize=(10,7))
# plt.title("Hair Color Train-Valid Loss Trend")
# plt.plot(y2_train_losses, color='green', label='Hair Color Training Loss')
# plt.plot(y2_valid_losses, color='blue', label='Hair Color Validation Loss')
# plt.legend(frameon=False)
# plt.xlabel("epochs")
# plt.ylabel("Hair Color Loss")
# plt.savefig(saving_path + "hair_color_loss_plot.png")
# plt.show()

In [102]:
# plt.figure(figsize=(10,7))
# plt.title("Gender & Hair Color Accuracy Trend")
# plt.plot(test_y1_acc, color='green', label='Gender Test set Accuracy')
# plt.plot(test_y2_acc, color='blue', label='Hair Color Test set Accuracy')
# plt.legend(frameon=False)
# plt.xlabel("epochs")
# plt.ylabel("Accuracy")
# plt.savefig(saving_path + "accuracy_test_plot.png")
# plt.show()

In [103]:
# first_batch = next(iter(test_loader))
# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.title("Test Images")
# plt.imshow(np.transpose(vutils.make_grid(first_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [104]:
# pics = first_batch[0].to(device)[:64]
# reconst_batch, y1 = netAE(pics)
# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.title("Reconstructed Images")
# plt.imshow(np.transpose(vutils.make_grid(reconst_batch, padding=2, normalize=True).cpu(),(1,2,0)))